In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import umap

In [ ]:
# Carica il dataset da dividere
untouched = pd.read_csv("BRCA_FINISHED_DATASET_WITH_VEP.csv")

# PCA

In [ ]:
df = untouched.copy()

In [ ]:
# Applica la PCA sulle colonne specificate
pca = PCA(n_components=1)
df["HGVS"] = pca.fit_transform(df[["HGVSG_SOST", "HGVSG_INS", "HGVSG_DEL"]])
pca.explained_variance_ratio_

In [ ]:
df['PCA'] = pca.fit_transform(df[["LABEL", "SESSO", "HGVS"]]) # , "TYPE_SOMATIC"]])
pca.explained_variance_ratio_

In [ ]:
#moltiplico la colonna per 10 per avere numeri interi
df['PCA'] = df['PCA'] * 100
df['PCA'] = df['PCA'].astype(int)

In [ ]:
df["PCA"].value_counts()

In [ ]:
# Esegui la divisione stratificata
X_train, X_test, y_train, y_test = train_test_split(df.drop('LABEL', axis=1), df['LABEL'], test_size=0.2, random_state=25, stratify=df['PCA'])

In [ ]:
# calcolo la percentuale di ogni classe nel train set
y_train.value_counts(normalize=True) * 100

In [ ]:
# calcolo la percentuale di ogni classe nel train set
y_train.value_counts(normalize=True) * 100

In [ ]:
y_test.value_counts(normalize=True) * 100

In [ ]:
#droppo la colonna PCA
X_train = X_train.drop('PCA', axis=1)
X_test = X_test.drop('PCA', axis=1)

#droppo la colonna HGVS
X_train = X_train.drop('HGVS', axis=1)
X_test = X_test.drop('HGVS', axis=1)

In [ ]:
# salvo i file
X_train.to_csv("X_train_PCA.csv", index=False)
X_test.to_csv("X_test_PCA.csv", index=False)
y_train.to_csv("y_train_PCA.csv", index=False)
y_test.to_csv("y_test_PCA.csv", index=False)

# UMAP

In [ ]:
df = untouched.copy()

In [ ]:
tha_umap = umap.UMAP(n_components=1)

In [ ]:
df["HGVS"] =tha_umap.fit_transform(df[["HGVSG_SOST", "HGVSG_INS", "HGVSG_DEL"]])

In [ ]:
df["umap"] = tha_umap.fit_transform(df[["LABEL", "SESSO", "HGVS"]])

In [ ]:
# per non avere valori unici utilizzo una knn per raggruppare i valori simili e poi li imputo con la media
imputer = KNNImputer()
#sostituisco i valori unici con nan
valori_unici = df['umap'].value_counts()[df['umap'].value_counts() == 1].index
df['umap'] = df['umap'].replace(valori_unici, np.nan)
#imputo i valori nan con la media
df['umap'] = imputer.fit_transform(df[['umap']])

In [ ]:
# df["umap"] = StandardScaler().fit_transform(df[["umap"]])
# df = df.sort_index()



df["umap"] = df["umap"] * 100
df["umap"] = df["umap"].astype(int)


# # se il value_counts() è uguale a 1, allora non è possibile fare la stratificazione, lo sostituisco quindi con il valore più vicino
# for i in df["umap"].value_counts().index:
#     if df["umap"].value_counts()[i] == 1:
#         #aumento il valore i di 1 e lo sostituisco
#         df["umap"].replace(i, i+1, inplace=True)
#         print("Sostituito il valore", i)


df["umap"].value_counts()

In [ ]:
# Esegui la divisione stratificata
X_train, X_test, y_train, y_test = train_test_split(df.drop('LABEL', axis=1), df['LABEL'], test_size=0.2, stratify=df['umap'])

In [ ]:
# calcolo la percentuale di ogni classe nel train set
y_train.value_counts(normalize=True) * 100

In [ ]:
y_test.value_counts(normalize=True) * 100

In [ ]:
#droppo la colonna umap
X_train = X_train.drop('umap', axis=1)
X_test = X_test.drop('umap', axis=1)

#droppo la colonna HGVS
X_train = X_train.drop('HGVS', axis=1)
X_test = X_test.drop('HGVS', axis=1)

In [ ]:
# salvo i file
X_train.to_csv("X_train_umap.csv", index=False)
X_test.to_csv("X_test_umap.csv", index=False)
y_train.to_csv("y_train_umap.csv", index=False)
y_test.to_csv("y_test_umap.csv", index=False)